# 🧠 Using MOISE+MARL in Overcooked-AI

This notebook demonstrates how to use the **MOISE+MARL framework** to enforce **organizational roles and constraints** in the Overcooked-AI environment using **script-based role logic**.

## 1. Install and Import Dependencies

In [ ]:
# !pip install marllib
# !pip install -e mma_wrapper/  # assuming your wrapper is a local editable package

In [ ]:
import numpy as np
import gym
from marllib import marl
from mma_wrapper.label_manager import label_manager
from mma_wrapper.organizational_model import (
    organizational_model, structural_specifications,
    functional_specifications, deontic_specifications,
    deontic_specification, time_constraint_type
)
from mma_wrapper.organizational_specification_logic import role_logic

## 2. Define the `label_manager` for Overcooked

In [ ]:
from collections import OrderedDict

def decompose_feature_vector(feature_vector, num_agents=2, num_pots=2):
    index = 0
    features = OrderedDict()
    features["orientation"] = feature_vector[index:index+4]; index += 4
    features["held_object"] = feature_vector[index:index+4]; index += 4
    for obj in ["onion", "tomato", "dish", "soup", "serving", "empty_counter"]:
        features[f"dist_{obj}"] = feature_vector[index:index+2]; index += 2
    features["soup_num_onions"] = feature_vector[index]; index += 1
    features["soup_num_tomatoes"] = feature_vector[index]; index += 1
    for pot_idx in range(num_pots):
        for key in ["exists", "is_empty", "is_full", "is_cooking", "is_ready",
                    "num_onions", "num_tomatoes", "cook_time"]:
            features[f"pot_{pot_idx}_{key}"] = feature_vector[index]; index += 1
        features[f"pot_{pot_idx}_dist"] = feature_vector[index:index+2]; index += 2
    for d in range(4):
        features[f"wall_{d}"] = feature_vector[index]; index += 1
    length_other = (num_agents - 1) * (num_pots * 10 + 26)
    features["other_player_features"] = feature_vector[index:index+length_other]; index += length_other
    features["relative_distances_to_others"] = feature_vector[index:index+2*(num_agents-1)]; index += 2*(num_agents-1)
    features["absolute_position"] = feature_vector[index:index+2]
    return dict(features)

In [ ]:
class OvercookedLabelManager(label_manager):
    def __init__(self, action_space=None, observation_space=None):
        super().__init__(action_space, observation_space)
        self.action_encode = {"up": 0, "down": 1, "right": 2, "left": 3, "nothing": 4, "interact": 5}
        self.action_decode = {v: k for k, v in self.action_encode.items()}

    def one_hot_decode_observation(self, observation, agent=None):
        return decompose_feature_vector(observation)

    def one_hot_encode_action(self, action, agent=None):
        return self.action_encode[action]

    def one_hot_decode_action(self, action, agent=None):
        return self.action_decode[action]

## 3. Define Script-Based Role Logic

In [ ]:
def primary_fun(trajectory, observation, agent_name, label_manager):
    obs = label_manager.one_hot_decode_observation(observation)
    orientation = obs["orientation"]
    held_object = np.array(obs["held_object"])
    dist_dish = obs["dist_dish"]
    dist_onion = obs["dist_onion"]
    dist_empty_counter = obs["dist_empty_counter"]
    facing_walls = [obs[f"wall_{d}"] for d in range(4)]

    # Sample simplified logic
    if held_object[0] == 1:
        return label_manager.one_hot_encode_action("interact")
    elif np.all(held_object == 0):
        return label_manager.one_hot_encode_action("right")
    return label_manager.one_hot_encode_action("nothing")

## 4. Create the Organizational Model

In [ ]:
label_mngr = OvercookedLabelManager()
org_model = organizational_model(
    structural_specifications(
        roles={
            "role_primary": role_logic(label_manager=label_mngr).registrer_script_rule(primary_fun),
            "role_secondary": role_logic(label_manager=label_mngr).registrer_script_rule(primary_fun)
        },
        role_inheritance_relations={},
        root_groups={}
    ),
    functional_specifications=functional_specifications(
        goals={}, social_scheme={}, mission_preferences=[]
    ),
    deontic_specifications=deontic_specifications(
        obligations=[
            deontic_specification("role_primary", ["agent_0"], [], time_constraint_type.ANY),
            deontic_specification("role_secondary", ["agent_1"], [], time_constraint_type.ANY)
        ],
        permissions=[]
    )
)

## 5. Create and Wrap the Environment

In [ ]:
env = marl.make_env(
    environment_name="overcooked",
    map_name="asymmetric_advantages",
    organizational_model=org_model
)

## 6. Define Algorithm and Train

In [ ]:
algo = marl.algos.mappo(hyperparam_source="test")
model = marl.build_model(env, algo, {"core_arch": "mlp", "encode_layer": "128-256"})

# Optional: Uncomment to train
# algo.fit(env, model, stop={"timesteps_total": 1e6})

## 7. Visualize & Analyze with TEMM

In [ ]:
algo.render(env, model,
    restore_path={
        "params_path": "path/to/params.json",
        "model_path": "path/to/checkpoint/checkpoint-XX",
        "render_env": True
    },
    enable_temm=True,
    local_mode=True,
    share_policy="group"
)

## ✅ Conclusion
This notebook showed how to:
- Define role logic using script-based rules
- Integrate those rules into MOISE+MARL
- Train and visualize behavior in Overcooked-AI
- Use the **TEMM method** to analyze organizational fit